In [4]:
import pandas as pd
import numpy as np
import copy

from datetime import datetime, timedelta, time

In [5]:
from solver import Solver

In [6]:
slvr = Solver('data/flights.json', 'data/crew.json')

slvr.flights

c:\robots\aircrew_assignment\solver.py:26: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  self.flights['dep'] = pd.to_datetime(self.flights['dep']).dt.time
c:\robots\aircrew_assignment\solver.py:27: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  self.flights['arr'] = pd.to_datetime(self.flights['arr']).dt.time


,day,id,origin,dest,dep,arr,duration,type
0,1,D1_F006,BCN,FRA,06:00:00,07:00:00,1,A319
1,1,D1_F003,LHR,BCN,06:00:00,08:00:00,2,A319
2,1,D1_F005,CDG,LHR,07:00:00,08:00:00,1,A319
3,1,D1_F016,AMS,MXP,07:00:00,08:00:00,1,A320
4,1,D1_F019,FRA,CDG,07:00:00,08:00:00,1,A320
...,...,...,...,...,...,...,...,...
245,5,D5_F020,CDG,FRA,18:00:00,19:00:00,1,A320
246,5,D5_F024,FRA,AMS,18:00:00,20:00:00,2,A320
247,5,D5_F008,AMS,LHR,19:00:00,20:00:00,1,B737
248,5,D5_F037,CDG,FRA,19:00:00,20:00:00,1,A320


In [7]:
slvr.crew

,id,name,role,base,qualified
0,C0,Brian Benson,captain,DUB,"[B737, A320]"
1,C1,Jacob Webb,captain,LHR,"[A319, A320]"
2,C2,Lisa Moore,captain,FRA,[A319]
3,C3,Timothy Combs,captain,FRA,[A320]
4,C4,Jacqueline Bailey,captain,LHR,[B737]
...,...,...,...,...,...
95,FO45,Krystal Todd,first_officer,FRA,[A319]
96,FO46,William Holmes,first_officer,MXP,"[B737, A319, A320]"
97,FO47,Michelle Lynn,first_officer,LHR,[B737]
98,FO48,Virginia Young,first_officer,LHR,[B737]


In [8]:
slvr.initial_assignment()

slvr.assignment

{'D1_F006': {'captain': 'C34', 'first_officer': 'FO26', 'dead_heading': []},
 'D1_F003': {'captain': 'C40', 'first_officer': 'FO36', 'dead_heading': []},
 'D1_F005': {'captain': 'C16', 'first_officer': 'FO43', 'dead_heading': []},
 'D1_F016': {'captain': 'C17', 'first_officer': 'FO38', 'dead_heading': []},
 'D1_F019': {'captain': 'C3', 'first_officer': 'FO4', 'dead_heading': []},
 'D1_F026': {'captain': 'C46', 'first_officer': 'FO25', 'dead_heading': []},
 'D1_F039': {'captain': 'C14', 'first_officer': 'FO15', 'dead_heading': []},
 'D1_F044': {'captain': 'C16', 'first_officer': 'FO41', 'dead_heading': []},
 'D1_F010': {'captain': 'C3', 'first_officer': 'FO12', 'dead_heading': []},
 'D1_F041': {'captain': 'C5', 'first_officer': 'FO44', 'dead_heading': []},
 'D1_F045': {'captain': 'C29', 'first_officer': 'FO35', 'dead_heading': []},
 'D1_F018': {'captain': 'C41', 'first_officer': 'FO32', 'dead_heading': []},
 'D1_F032': {'captain': 'C45', 'first_officer': 'FO5', 'dead_heading': []},
 'D1

In [9]:
slvr.initial_crew_state()
slvr.crew_state 

{'C0': [{'day': 5,
   'flight': 'D5_F044',
   'role': 'captain',
   'origin': 'DUB',
   'destination': 'AMS',
   'depart': datetime.time(19, 0),
   'arrive': datetime.time(21, 0)}],
 'C1': [{'day': 3,
   'flight': 'D3_F007',
   'role': 'captain',
   'origin': 'LHR',
   'destination': 'CPH',
   'depart': datetime.time(8, 0),
   'arrive': datetime.time(10, 0)},
  {'day': 3,
   'flight': 'D3_F022',
   'role': 'captain',
   'origin': 'LHR',
   'destination': 'FRA',
   'depart': datetime.time(12, 0),
   'arrive': datetime.time(14, 0)},
  {'day': 4,
   'flight': 'D4_F027',
   'role': 'captain',
   'origin': 'LHR',
   'destination': 'CDG',
   'depart': datetime.time(8, 0),
   'arrive': datetime.time(9, 0)},
  {'day': 4,
   'flight': 'D4_F038',
   'role': 'captain',
   'origin': 'LHR',
   'destination': 'AMS',
   'depart': datetime.time(9, 0),
   'arrive': datetime.time(10, 0)},
  {'day': 4,
   'flight': 'D4_F015',
   'role': 'captain',
   'origin': 'LHR',
   'destination': 'BCN',
   'depart':

In [10]:
total_assignment_cost, diagnostics = slvr.compute_assignment_cost(slvr.assignment)
print(total_assignment_cost)

13700


In [11]:
total_crew_cost, diagnostics = slvr.compute_crew_cost(slvr.crew_state)
print(total_crew_cost)

584300


In [12]:
destroy_repair_strength = 0.2
num_destroy_operators = 2
num_repair_operators = 1

num_to_destroy = int(len(slvr.flights) * destroy_repair_strength)

In [13]:
d_assignment, d_crew_state, flights_destroyed, affected_crew_list = slvr.destroy(num_to_destroy)

r_assignment, r_crew_state = slvr.repair()
r_assignment_cost, _ = slvr.compute_assignment_cost(slvr.r_assignment)
r_crew_cost, _ = slvr.compute_crew_cost(slvr.r_crew_state)

Destroy - using operator 1 => target overlapping flights
Repair - using operator 0 => random repair


In [14]:
print(f"{total_assignment_cost}, {total_crew_cost} => {total_assignment_cost+total_crew_cost}, \n {r_assignment_cost}, {r_crew_cost} => {r_assignment_cost+r_crew_cost}")

13700, 584300 => 598000, 
 38200, 553800 => 592000


In [15]:
r_assignment

{'D1_F006': {'captain': 'C34', 'first_officer': 'FO26', 'dead_heading': []},
 'D1_F003': {'captain': 'C40', 'first_officer': 'FO36', 'dead_heading': []},
 'D1_F005': {'captain': 'C16', 'first_officer': 'FO43', 'dead_heading': []},
 'D1_F016': {'captain': 'C17', 'first_officer': 'FO38', 'dead_heading': []},
 'D1_F019': {'captain': 'C3', 'first_officer': 'FO4', 'dead_heading': []},
 'D1_F026': {'captain': 'C46', 'first_officer': 'FO25', 'dead_heading': []},
 'D1_F039': {'captain': 'C14', 'first_officer': 'FO15', 'dead_heading': []},
 'D1_F044': {'captain': 'C16', 'first_officer': 'FO41', 'dead_heading': []},
 'D1_F010': {'captain': 'C3', 'first_officer': 'FO12', 'dead_heading': []},
 'D1_F041': {'captain': 'C5', 'first_officer': 'FO41', 'dead_heading': []},
 'D1_F045': {'captain': 'C29', 'first_officer': 'FO35', 'dead_heading': []},
 'D1_F018': {'captain': 'C41', 'first_officer': 'FO32', 'dead_heading': []},
 'D1_F032': {'captain': 'C45', 'first_officer': 'FO5', 'dead_heading': []},
 'D1

In [16]:
r_crew_state

{'C0': [{'day': 2,
   'flight': 'D2_F008',
   'role': 'captain',
   'origin': 'MXP',
   'destination': 'DUB',
   'depart': datetime.time(10, 0),
   'arrive': datetime.time(11, 0),
   'duration': 1},
  {'day': 5,
   'flight': 'D5_F026',
   'role': 'captain',
   'origin': 'LHR',
   'destination': 'CDG',
   'depart': datetime.time(8, 0),
   'arrive': datetime.time(9, 0),
   'duration': 1},
  {'day': 5,
   'flight': 'D5_F044',
   'role': 'captain',
   'origin': 'DUB',
   'destination': 'AMS',
   'depart': datetime.time(19, 0),
   'arrive': datetime.time(21, 0)}],
 'C1': [{'day': 1,
   'flight': 'D1_F009',
   'role': 'captain',
   'origin': 'AMS',
   'destination': 'CDG',
   'depart': datetime.time(13, 0),
   'arrive': datetime.time(14, 0),
   'duration': 1},
  {'day': 3,
   'flight': 'D3_F007',
   'role': 'captain',
   'origin': 'LHR',
   'destination': 'CPH',
   'depart': datetime.time(8, 0),
   'arrive': datetime.time(10, 0)},
  {'day': 3,
   'flight': 'D3_F022',
   'role': 'captain',
  